In [1]:
import torch
from torch import nn
from torch.nn import init
import numpy as np
import pandas as pd
import sys
sys.path.append('../../')
import ML as ml

In [2]:
df = pd.read_csv('trainset.csv')
df.status[df['status']==-1]=0
df

/home/kong123/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,u,e,a_p,inc,status
0,0.11,0.98,4.16,0.698132,0
1,0.03,0.67,2.81,0.523599,0
2,0.48,0.92,2.21,1.396263,0
3,0.33,0.43,3.77,2.268928,1
4,0.47,0.26,5.51,2.268928,1
...,...,...,...,...,...
999995,0.38,0.12,5.34,1.221730,1
999996,0.22,0.38,3.54,1.570796,1
999997,0.39,0.22,1.80,2.268928,0
999998,0.09,0.64,3.96,2.967060,1


In [3]:
n_train = 900000
train_set = df.iloc[:n_train]
val_set = df.iloc[n_train:]
all_features = df.iloc[:,:-1]

In [4]:
s =set(all_features['inc'])
inc = list(s)
inc.sort()
for i in range(len(inc)):
    all_features.inc[all_features['inc']==inc[i]]='%d'%i

In [5]:
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
# 标准化后，每个特征的均值变为0，所以可以直接用0来替换缺失值
all_features[numeric_features] = all_features[numeric_features].fillna(0)

In [6]:
all_features = pd.get_dummies(all_features, dummy_na=False)
all_features

,u,e,a_p,inc_0,inc_1,inc_10,inc_11,inc_12,inc_13,inc_14,...,inc_17,inc_18,inc_2,inc_3,inc_4,inc_5,inc_6,inc_7,inc_8,inc_9
0,-1.007467,1.679820,0.314535,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,-1.561635,0.605801,-0.723102,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,1.555560,1.471945,-1.184274,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0.516495,-0.225698,0.014773,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1.486289,-0.814676,1.352171,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0.862850,-1.299717,1.221506,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
999996,-0.245486,-0.398926,-0.162010,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
999997,0.932121,-0.953259,-1.499408,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
999998,-1.146009,0.501864,0.160811,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [7]:
train_features = torch.tensor(all_features[:n_train].values, dtype=torch.float)
val_features = torch.tensor(all_features[n_train:].values, dtype=torch.float)
train_labels = torch.tensor(train_set.status.values, dtype=torch.long)
val_labels = torch.tensor(val_set.status.values, dtype=torch.long)

In [8]:
num_inputs, num_outputs, num_hiddens = 22, 2, 256

net = nn.Sequential(
        ml.FlattenLayer(),
        nn.Linear(num_inputs, num_hiddens),
        nn.ReLU(),
        nn.Linear(num_hiddens, num_hiddens),
        nn.ReLU(),
        nn.Linear(num_hiddens, num_hiddens),
        nn.ReLU(),
        nn.Linear(num_hiddens, num_hiddens),
        nn.ReLU(),
        nn.Linear(num_hiddens, num_outputs), 
        )

for params in net.parameters():
    init.normal_(params, mean=0, std=0.01)


In [9]:
batch_size = 512
dataset = torch.utils.data.TensorDataset(train_features, train_labels)
train_iter = torch.utils.data.DataLoader(dataset, batch_size, shuffle=True)
valset = torch.utils.data.TensorDataset(val_features, val_labels)
val_iter = torch.utils.data.DataLoader(valset, batch_size, shuffle=True)

In [10]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.5, momentum=0.8)
num_epochs = 100

In [11]:
train_ls, test_ls = ml.train(net, train_iter, val_iter, loss, num_epochs, batch_size, None, None, optimizer)

epoch 1, loss 0.0005, train acc 0.858, test acc 0.955
epoch 2, loss 0.0002, train acc 0.961, test acc 0.963
epoch 3, loss 0.0002, train acc 0.963, test acc 0.966
epoch 4, loss 0.0002, train acc 0.965, test acc 0.966
epoch 5, loss 0.0002, train acc 0.966, test acc 0.966
epoch 6, loss 0.0002, train acc 0.967, test acc 0.964
epoch 7, loss 0.0002, train acc 0.968, test acc 0.968
epoch 8, loss 0.0001, train acc 0.968, test acc 0.968
epoch 9, loss 0.0001, train acc 0.969, test acc 0.969
epoch 10, loss 0.0001, train acc 0.969, test acc 0.970
epoch 11, loss 0.0001, train acc 0.970, test acc 0.967
epoch 12, loss 0.0001, train acc 0.971, test acc 0.970
epoch 13, loss 0.0001, train acc 0.971, test acc 0.973
epoch 14, loss 0.0001, train acc 0.972, test acc 0.971
epoch 15, loss 0.0001, train acc 0.972, test acc 0.973
epoch 16, loss 0.0001, train acc 0.973, test acc 0.973
epoch 17, loss 0.0001, train acc 0.973, test acc 0.971
epoch 18, loss 0.0001, train acc 0.973, test acc 0.972
epoch 19, loss 0.00

In [12]:
PATH = "./net.pt"
torch.save(net, PATH)